In [13]:
import sympy as sp
import numpy as np

# Define symbols for state variables and control inputs
x, y, z = sp.symbols('x y z')
xdot, ydot, zdot = sp.symbols('xdot ydot zdot')
theta, phi, t = sp.symbols('theta phi t')
psi = sp.symbols('psi')
m, g = sp.symbols('m g')
Kvx, Kvy = sp.symbols('Kvx Kvy')

# Define the state vector
state = sp.Matrix([x, y, z, xdot, ydot, zdot])

# Define the control vector
control = sp.Matrix([theta, phi, t])

# State transition equations (dynamics of the drone)
x_acc = -Kvx/m * xdot + t/m*sp.sin(theta)
y_acc = -Kvy/m * ydot + t/m*sp.sin(phi)
z_acc = t/m*sp.cos(theta)*sp.cos(phi) - g

# Rotation matrix from reference frame to world frame
R = sp.Matrix([[sp.cos(psi), sp.sin(psi), 0],
               [-sp.sin(psi), sp.cos(psi), 0],
               [0, 0, 1]])

# World frame accelerations
acc_world = R * sp.Matrix([x_acc, y_acc, z_acc])

# Assuming a time step 'dt' for discretization
dt = sp.symbols('dt')

# Discretize the state transition equations using the Euler method
# New state is the old state plus the derivative (from f) times dt
x_next = x + xdot * dt
y_next = y + ydot * dt
z_next = z + zdot * dt
xdot_next = xdot + acc_world[0] * dt
ydot_next = ydot + acc_world[1] * dt
zdot_next = zdot + acc_world[2] * dt

# Discrete state transition function f_discrete
f_discrete = sp.Matrix([x_next, y_next, z_next, xdot_next, ydot_next, zdot_next])

# Compute the Jacobian of the discrete state transition function with respect to the state
F_discrete = f_discrete.jacobian(state)

# Compute the Jacobian of the discrete state transition function with respect to the control input
G_discrete = f_discrete.jacobian(control)

F_discrete, G_discrete

(Matrix([
 [1, 0, 0,                     dt,                      0,  0],
 [0, 1, 0,                      0,                     dt,  0],
 [0, 0, 1,                      0,                      0, dt],
 [0, 0, 0, -Kvx*dt*cos(psi)/m + 1,     -Kvy*dt*sin(psi)/m,  0],
 [0, 0, 0,      Kvx*dt*sin(psi)/m, -Kvy*dt*cos(psi)/m + 1,  0],
 [0, 0, 0,                      0,                      0,  1]]),
 Matrix([
 [                          0,                           0,                                                0],
 [                          0,                           0,                                                0],
 [                          0,                           0,                                                0],
 [ dt*t*cos(psi)*cos(theta)/m,    dt*t*sin(psi)*cos(phi)/m, dt*(sin(phi)*sin(psi)/m + sin(theta)*cos(psi)/m)],
 [-dt*t*sin(psi)*cos(theta)/m,    dt*t*cos(phi)*cos(psi)/m, dt*(sin(phi)*cos(psi)/m - sin(psi)*sin(theta)/m)],
 [-dt*t*sin(theta)*cos(phi)/m, -dt*t*si

In [14]:
F_discrete.subs([(x, 0), (y, 0), (z, 0), (xdot, 0), (ydot, 0), (zdot, 0), (theta, 0), (phi, 0), (t, 0), (psi, 0), (m, 1), (g, 9.81), (Kvx, 1), (Kvy, 1), (dt, 0.1)])

Matrix([
[1, 0, 0, 0.1,   0,   0],
[0, 1, 0,   0, 0.1,   0],
[0, 0, 1,   0,   0, 0.1],
[0, 0, 0, 0.9,   0,   0],
[0, 0, 0,   0, 0.9,   0],
[0, 0, 0,   0,   0,   1]])

In [15]:
G_discrete.subs([(x, 0), (y, 0), (z, 0), (xdot, 0), (ydot, 0), (zdot, 0), (theta, np.pi), (phi, 0), (t, 0), (psi, 0), (m, 1), (g, 9.81), (Kvx, 1), (Kvy, 1), (dt, 0.1)])

Matrix([
[0, 0,                    0],
[0, 0,                    0],
[0, 0,                    0],
[0, 0, 1.22464679914735e-17],
[0, 0,                    0],
[0, 0,                 -0.1]])

In [5]:
# Define the measurement vector, which directly measures position
# For an Optitrack system, the measurements would be direct position readings
z_measured = sp.Matrix([x, y, z])

# Define the measurement function h, which maps the state to the measurement
# In this case, it's a direct observation of the position states
h = z_measured

# Compute the Jacobian of the measurement function with respect to the state
H = h.jacobian(state)

# Display the measurement function Jacobian
H

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0]])

In [7]:
import numpy as np

class ExtendedKalmanFilter:
    def __init__(self, dt, state_dim, control_dim, measure_dim):
        self.dt = dt  # time step
        self.state_dim = state_dim  # dimension of the state vector
        self.control_dim = control_dim  # dimension of the control vector
        self.measure_dim = measure_dim  # dimension of the measurement vector

        # Initialize state estimate and covariance matrix
        self.x_hat = np.zeros(state_dim)  # state estimate
        self.P = np.eye(state_dim)  # estimate covariance matrix

        # Process and measurement noise covariance matrices
        self.Q = np.eye(state_dim)  # process noise covariance
        self.R = np.eye(measure_dim) * 0.001  # measurement noise covariance (small values due to high precision)

        # Placeholder for the Jacobians
        self.F = None  # state transition Jacobian
        self.H = np.eye(measure_dim)  # measurement Jacobian is identity since we have direct position measurement
        self.G = None  # control input Jacobian

    def predict(self, u):
        """
        Predict the state estimate and covariance using the control input 'u'
        :param u: control input vector
        """
        # Evaluate the state transition Jacobian F and control input Jacobian G for the current state
        F_num = np.array(F_discrete.subs(zip(state, self.x_hat)).subs(zip(control, u))).astype(float)
        G_num = np.array(G_discrete.subs(zip(state, self.x_hat)).subs(zip(control, u))).astype(float)

        # Update Jacobians in the class
        self.F = F_num
        self.G = G_num

        # State prediction using the discretized equations
        # Here you would use the discretized state transition equations
        # For simplicity, I'm using a placeholder for the state update, replace it with your discretized equations
        self.x_hat = self.x_hat + np.array([self.x_hat[3], self.x_hat[4], self.x_hat[5],
                                            -Kvx/self.x_hat[6] * self.x_hat[3] + u[2]/self.x_hat[6]*np.sin(u[0]),
                                            -Kvy/self.x_hat[6] * self.x_hat[4] + u[2]/self.x_hat[6]*np.sin(u[1]),
                                            u[2]/self.x_hat[6]*np.cos(u[0])*np.cos(u[1]) - g]) * self.dt

        # Covariance prediction
        self.P = self.F @ self.P @ self.F.T + self.G @ self.Q @ self.G.T

    def update(self, z):
        """
        Update the state estimate and covariance using the measurement 'z'
        :param z: measurement vector
        """

        # Measurement residual
        y = z - self.H @ self.x_hat

        # Kalman gain
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)

        # State update
        self.x_hat = self.x_hat + K @ y

        # Covariance update
        I = np.eye(self.state_dim)
        self.P = (I - K @ self.H) @ self.P

# Parameters for drone EKF (to be defined based on your system)
dt = 0.1  # time step
state_dim = 6  # state vector dimension: x, y, z, xdot, ydot, zdot
control_dim = 3  # control vector dimension: theta, phi, t
measure_dim = 3  # measurement vector dimension: x, y, z

# Constants (to be defined based on your system)
Kvx = 0.1  # drag coefficient in x-direction
Kvy = 0.1  # drag coefficient in y-direction
m = 1.0  # mass of the drone
g = 9.81  # acceleration due to gravity

# Create an instance of the EKF for the drone
drone_ekf = ExtendedKalmanFilter(dt, state_dim, control_dim, measure_dim)

# Example usage
# Define control input (theta, phi, thrust)
control_input = np.array([0.1, 0.1, 5.0])

# Predict step
drone_ekf.predict(control_input)

# Measurement vector (x, y, z positions)
measurement = np.array([1.0, 1.0, 1.0])

# Update step
drone_ekf.update(measurement)

# Estimated state after prediction and update
estimated_state = drone_ekf.x_hat
estimated_state_covariance = drone_ekf.P

IndexError: index 6 is out of bounds for axis 0 with size 6

In [ ]:
import numpy as np
import sympy as sp

class ExtendedKalmanFilter:
    def __init__(self, initial_state, initial_error_covariance, Ts, m, g, Kvx, Kvy):
        self.state_estimate = np.array(initial_state)
        self.error_covariance = np.array(initial_error_covariance)
        self.Ts = Ts  # Time step
        self.m = m    # Mass of the drone
        self.g = g    # Acceleration due to gravity
        self.Kvx = Kvx # Drag coefficient in x-direction
        self.Kvy = Kvy # Drag coefficient in y-direction

        # Define symbols for state variables and control inputs
        x, y, z, xdot, ydot, zdot = sp.symbols('x y z xdot ydot zdot')
        theta, phi, t = sp.symbols('theta phi t')
        psi = sp.symbols('psi')
        self.state_symbols = sp.Matrix([x, y, z, xdot, ydot, zdot])
        self.control_symbols = sp.Matrix([theta, phi, t])

        # Define the discrete state transition and measurement functions here
        # ...

    def compute_jacobians(self, control_input):
        # Evaluate F and G here based on the current state and control input
        # ...

    def predict(self, control_input):
        # Update F and G using current state and control input
        self.compute_jacobians(control_input)

        # State prediction using the discretized state transition function
        # ...

        # Covariance prediction
        # ...

    def update(self, measurement):
        # Measurement update using H (measurement Jacobian)
        # ...

        # Return the state estimate and the Kalman gain (if needed)
        return self.state_estimate, kalman_gain

# Parameters for drone EKF
Ts = 0.1  # Time step
initial_state = [0, 0, 0, 0, 0, 0]  # Example initial state
initial_error_covariance = np.eye(6)  # Initial error covariance
m = 1.0  # Mass of the drone
g = 9.81  # Gravity
Kvx = 0.1  # Drag coefficient in x-direction
Kvy = 0.1  # Drag coefficient in y-direction

# Create an instance of the EKF for the drone
drone_ekf = ExtendedKalmanFilter(initial_state, initial_error_covariance, Ts, m, g, Kvx, Kvy)

# Example usage
control_input = [0.1, 0.1, 5.0]  # Example control input (theta, phi, thrust)
measurement = [1.0, 1.0, 1.0]  # Example measurement (x, y, z positions)

# Predict and update steps
drone_ekf.predict(control_input)
estimated_state, kalman_gain = drone_ekf.update(measurement)
